In [ ]:
# Librerias.

import os
import pandas as pd
from tqdm import tqdm

### Generar DataFrame completo de la serie histórica.

Adjudicar los caladeros a los puertos.

In [25]:
# Cargar el DataFrame de cuota_pesquera.
df_cuota_pesquera = pd.read_csv(r'ruta a cuota_pesquera.csv')

# Definir el diccionario de mapeo Puerto -> Caladeros.
puerto_caladeros_dict = {
    'Adra': [43, 46, 48, 49, 50, 52, 53, 55, 58, 60, 66, 78, 95, 133],
    'Almería': [29, 32, 33, 34, 35, 38, 66, 73, 75, 77, 78, 79, 81],
    'Carboneras': [29],
    'Garrucha': [1, 4, 16, 20, 23],
    'La Atunara': [99, 103, 106, 107, 108, 110, 113, 114, 115, 118, 119, 121, 123, 124, 125, 126, 129],
    'La Caleta de Vélez': [64, 65, 67, 69, 70, 72, 80, 82, 85, 88, 89, 96, 98],
    'Marbella': [98, 99, 101, 102, 103],
    'Motril': [51, 54, 57, 58, 59, 63, 71, 74, 76, 128, 133],
    'Málaga': [40, 41, 42, 44, 45, 47, 56, 59, 61, 62, 68, 69, 116],
    'Roquetas de Mar': [29, 35, 39, 83, 84, 133]
}

# Asignar los caladeros correspondientes a cada puerto en df_cuota_pesquera.
df_cuota_pesquera['Caladeros'] = df_cuota_pesquera['Puerto'].map(puerto_caladeros_dict)

df_cuota_pesquera.drop(['CaladeroID'], axis=1, inplace=True)

# Guardar el resultado en un nuevo archivo CSV.
df_cuota_pesquera.to_csv('estado_actual.csv', index=False)

Unir al DataFrame los datos de temperatura, pH y clorofila.

In [2]:
df1 = pd.read_csv(r'ruta a estado_actual.csv')
df2 = pd.read_csv(r'ruta a resultados_prediccion_temperatura.csv')
df3 = pd.read_csv(r'ruta a resultados_prediccion_clorofila_con_pH.csv')

# Diccionario de los meses.
meses = {
    'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6,
    'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
}

# Mapear los meses en df_estado_actual.
df1['Mes'] = df1['Mes'].map(meses)

# Itera sobre cada año desde 2000 hasta 2024 con tqdm.
for año in tqdm(range(2000, 2025), desc="Procesando años"):
    # Filtra los datos para el año actual.
    df1_año = df1[df1['Año'] == año]
    df2_año = df2[df2['Año'] == año]
    df3_año = df3[df3['Año'] == año]
    
    # Unir los DataFrames para cada año y mes.
    df_merged = pd.merge(df1_año, df2_año, on=['Año', 'Mes'], how='outer')
    df_final = pd.merge(df_merged, df3_año, on=['Año', 'Mes'], how='outer')
    
    # Guardar el DataFrame resultante en un archivo CSV.
    nombre_archivo = f'datos_año_{año}.csv'
    df_final.to_csv(nombre_archivo, index=False)


Procesando años: 100%|██████████| 11/11 [06:55<00:00, 37.73s/it]


Unificar en un solo CSV todos los años.

In [ ]:
# Ruta de la carpeta que contiene los archivos CSV.
carpeta = r'ruta a datos_completos_por_año'

# Lista para almacenar los DataFrames de cada archivo CSV.
dfs = []

# Obtener la cantidad total de archivos en la carpeta.
total_archivos = len([archivo for archivo in os.listdir(carpeta) if archivo.endswith('.csv')])

# Iterar sobre cada archivo en la carpeta con tqdm.
for archivo in tqdm(os.listdir(carpeta), desc="Uniendo archivos", total=total_archivos):
    if archivo.endswith('.csv'):
        # Leer el archivo CSV y añadirlo a la lista.
        df = pd.read_csv(os.path.join(carpeta, archivo))
        dfs.append(df)

# Concatenar todos los DataFrames en uno solo.
df_evolucion_historica = pd.concat(dfs, ignore_index=True)

# Guardar el DataFrame resultante en un archivo CSV.
df_evolucion_historica.to_csv(r'D:\BOOTCAMP\datos_completos_por_año\evolucion_historica.csv', index=False)



En caso de que los recursos de Hardware del ordenador no permitan la ejecución debido a límites de memoria, esta acción se puede realizar mediante un pipeline en la plataforma Azure de Microsoft, debiendo de crear y activar una maquina virtual de almenos 32 GB de RAM debido al gran tamaño de los archivos.

Con el archivo finalizado, se puede descargar el archivo CSV y descargar el archivo en la memoria local.